## Master driver script for Tasks 1 - 3

In [1]:
# Master imports
# Train critic
from hw1 import Critic, Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment

import torch # Requried to create tensors to store all numerical values
import torch.nn as nn # Required for weight and bias tensors
import torch.nn.functional as F # Required for the activation functions
from torch.utils.data import TensorDataset, DataLoader, Dataset # How do we use them?
import pretty_midi

import numpy as np
root = '.'
midi_sav_dir = './maestro-v1.0.0-midi/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Global variables
batch_size = 50
seq_to_gen = 10000
seq_len = 51 # Hardcoded, actual lenght is 50, +1 to accomodate training composer
epoch = 10

midi_load = process_midi_seq(datadir=root, n=seq_to_gen, maxlen=(seq_len - 1)) # Master dataset to be used by both Composer and Critic
midi_load_bck = np.copy(midi_load)

Looking for files in: ./maestro-v1.0.0-midi/**/*.midi
1284


### Task 2 Train Composer

In [2]:
# Train Composer
conv_val = 0.001
conv_flag = False
loss = None
loader_composer = DataLoader(TensorDataset(torch.from_numpy(midi_load)), shuffle=True, batch_size=batch_size, num_workers=4) # Composer training dataset

cps_train = Composer(seq_len = (seq_len - 1)) # Loads composer model with default values
for i in range(epoch):
    for idx, x in enumerate(loader_composer):
        loss = cps_train.train(x[0].cuda(0).long())
        print(f"Epoch: {i}, Batch: {idx}, Loss: {loss.item()}")
        if (loss.item() < conv_val):
            print(f"convergence reached")
            conv_flag = True
        
        if(conv_flag):
            break # 
    
    if(conv_flag):
            print(f"Training complete with loss: {loss.item()}")
            break # 

##Save model
#* Activate to save model
# torch.save(cps.model.state_dict(), "az_cps.pth")

ComposerLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=382, bias=True)
  (softmax): Softmax(dim=1)
)
Loss in this bach: 5.961249828338623
Epoch: 0, Batch: 0, Loss: 5.961249828338623
Loss in this bach: 5.770368576049805
Epoch: 0, Batch: 1, Loss: 5.770368576049805
Loss in this bach: 5.624755382537842
Epoch: 0, Batch: 2, Loss: 5.624755382537842
Loss in this bach: 4.782129764556885
Epoch: 0, Batch: 3, Loss: 4.782129764556885
Loss in this bach: 5.368162155151367
Epoch: 0, Batch: 4, Loss: 5.368162155151367
Loss in this bach: 4.929294586181641
Epoch: 0, Batch: 5, Loss: 4.929294586181641
Loss in this bach: 4.646929740905762
Epoch: 0, Batch: 6, Loss: 4.646929740905762
Loss in this bach: 5.023732662200928
Epoch: 0, Batch: 7, Loss: 5.0

In [3]:
# Test composition of music
# cps_test = Composer(load_trained=True)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Will have access to this from hw1.py
# print()

# midi = cps_test.compose(n = 200)
# midi = seq2piano(midi)
# midi.write("midi_test.midi")

### Task 1: Train Critic

In [4]:
# Train Critic
loss = None
conv_flag = False
conv_val = 0.001

midi_load = np.copy(midi_load_bck)
# midi_load = process_midi_seq(datadir=root, n=seq_to_gen, maxlen=(seq_len - 1)) # Numpy array
# #print(f"midi_load shape: {midi_load.shape}")

# Split into training and testing dataset
#* Adopted from https://github.com/alizano94/CSC-7343-HW1/blob/main/HW1/main.py
#* Modified to conform to shape requirements

train_percent = 0.8
train_size = int(train_percent*midi_load.shape[0])

train_seq_data = midi_load[0:(int(train_size)), :-1]
test_seq_data = midi_load[(int(train_size)):,:-1]

train_labels = np.ones(train_seq_data.shape[0])
test_labels = np.ones(test_seq_data.shape[0])

# Create bad samples for training
for i in range(train_seq_data.shape[0]):
    rand_midi = random_piano(n = 250)
    rand_seq = piano2seq(rand_midi)
    rand_seq =  rand_seq[0:seq_len - 1]
    train_seq_data = np.vstack((train_seq_data, rand_seq))
    train_labels = np.append(train_labels,0)

# Create bad samples for testing
for i in range(test_seq_data.shape[0]):
    rand_midi = random_piano(n = 250)
    rand_seq = piano2seq(rand_midi)
    rand_seq =  rand_seq[0:seq_len - 1]
    test_seq_data = np.vstack((test_seq_data, rand_seq))
    test_labels = np.append(test_labels,0)

x_train_tensors = torch.Tensor(train_seq_data)
y_train_tensors = torch.Tensor(train_labels)

x_test_tensors = torch.Tensor(test_seq_data)
y_test_tensors = torch.Tensor(test_labels)

#* Debug
print(x_train_tensors.shape)
# print(y_train_tensors.shape)
print(x_test_tensors.shape)
# print(y_test_tensors.shape)
# print(x_train_tensors_final)

dataset = TensorDataset(x_train_tensors, y_train_tensors)
test_dataset = TensorDataset(x_test_tensors, y_test_tensors)

loader_critic = DataLoader(dataset, batch_size=batch_size, shuffle=True) # Critic training
loader_critic_tester = DataLoader(test_dataset, batch_size=batch_size, shuffle=True) # One sequence at a time


torch.Size([18416, 50])
torch.Size([4604, 50])


In [5]:
# Train Critic
ctr_train = Critic()

for i in range(epoch):
    for idx, x in enumerate(loader_critic):
        loss = ctr_train.train((x[0], x[1])) # Send the tuple directly 
        print(f"Epoch: {i}, Batch: {idx}, Loss: {loss.item()}")
        #* Critic's error gets very very low. Either its an overfit or the chosen model is really good

##Save model
# torch.save(ctr.model.state_dict(), "az_crt.pth")

CriticLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 0, Batch: 0, Loss: 0.6943409442901611
Epoch: 0, Batch: 1, Loss: 0.6943870186805725
Epoch: 0, Batch: 2, Loss: 0.6858218312263489
Epoch: 0, Batch: 3, Loss: 0.6836132407188416
Epoch: 0, Batch: 4, Loss: 0.6898441910743713
Epoch: 0, Batch: 5, Loss: 0.6845448017120361
Epoch: 0, Batch: 6, Loss: 0.6714575290679932
Epoch: 0, Batch: 7, Loss: 0.6589372158050537
Epoch: 0, Batch: 8, Loss: 0.6611936688423157
Epoch: 0, Batch: 9, Loss: 0.6397061347961426
Epoch: 0, Batch: 10, Loss: 0.6315664649009705
Epoch: 0, Batch: 11, Loss: 0.6043264865875244
Epoch: 0, Batch: 12, Loss: 0.6228228807449341
Epoch: 0, Batch: 13, Loss: 0.6089800596237183
Epoch: 0

In [6]:
# Test trained Critic on test dataset
# crt2 = Critic(load_trained=True)
# crt2.model.to(device)
# crt2.model.eval() # Puts model in evaluation mode

# # Cycle through the batches in the test dataset
# for idx, x in enumerate(loader_critic_tester):
#     # Cycle through each sample one at a time
#     x_batch = x[0] # All the sequences
#     y_batch = x[1] # All the test labels
#     running_score = []
#     for idx, x_in in enumerate(x_batch):
#         score = crt2.score(x_in.cuda(0).long()) # Only the music sequence
#         print(f"This sample score: -- {score}, expected: {y_batch[idx]}")
#         running_score.append(score)
#     print(len(running_score))
#     print(f"Good music probability -- {np.mean(np.array(running_score))}")
    
#     break

### Task 3: Average score for Critic vs Composer

In [7]:
%%capture
# Load models from internet and initialize to evaluation mode
cps = Composer(load_trained=True)
cps.model.eval()

crt = Critic(load_trained=True)
crt.model.to(device)
crt.model.eval() 

# Load models by downloading weights from internet
num_exp = 100
lss_scores = []

for i in range(num_exp):
    f_name = f"midi_test_" + str(i) + f".midi"
    pth = midi_sav_dir + "/" + f_name
    midi = cps.compose(n = 200) # Numpy array of midi notes

    midi_torch = torch.LongTensor(midi) # Input to the Critic LSTM
    # print(midi_torch.shape[0])
    score = crt.score(midi_torch)
    print(f"This music score: {score}")

    lss_scores.append(score)

    # Write this music
    midi = seq2piano(midi)
    midi.write(pth)
    print()

DEBUG:up without down for pitch 41 at time 4
DEBUG:up without down for pitch 46 at time 4
DEBUG:up without down for pitch 94 at time 4
DEBUG:up without down for pitch 38 at time 4
DEBUG:up without down for pitch 81 at time 4
DEBUG:consecutive downs for pitch 61 at time 4 and 4
DEBUG:up without down for pitch 76 at time 4
DEBUG:up without down for pitch 72 at time 5
DEBUG:up without down for pitch 41 at time 5
DEBUG:up without down for pitch 39 at time 5
DEBUG:up without down for pitch 50 at time 5
DEBUG:up without down for pitch 44 at time 5
DEBUG:up without down for pitch 50 at time 5
DEBUG:up without down for pitch 75 at time 5
DEBUG:up without down for pitch 88 at time 5
DEBUG:up without down for pitch 39 at time 5
DEBUG:up without down for pitch 44 at time 6
DEBUG:up without down for pitch 57 at time 7
DEBUG:consecutive downs for pitch 61 at time 4 and 7
DEBUG:consecutive downs for pitch 61 at time 4 and 7
DEBUG:up without down for pitch 87 at time 7
DEBUG:up without down for pitch

In [ ]:
print()
print(f"Average composer vs critic experiment: {np.mean(np.array(lss_scores))}")

# Test composition of music
cps_test = Composer(load_trained=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Will have access to this from hw1.py
print()

midi = cps_test.compose(n = 200)
midi = seq2piano(midi)
midi.write("midi_test.midi")
# USE AN ONLINE TOOL TO CONVERT IT TO MP3

DEBUG:up without down for pitch 57 at time 3
DEBUG:up without down for pitch 83 at time 3
DEBUG:up without down for pitch 45 at time 3
DEBUG:up without down for pitch 76 at time 4
DEBUG:up without down for pitch 70 at time 4
DEBUG:up without down for pitch 38 at time 4
DEBUG:up without down for pitch 35 at time 4
DEBUG:up without down for pitch 89 at time 4
DEBUG:up without down for pitch 63 at time 4
DEBUG:up without down for pitch 47 at time 4
DEBUG:up without down for pitch 46 at time 5
DEBUG:up without down for pitch 40 at time 5
DEBUG:up without down for pitch 66 at time 5
DEBUG:up without down for pitch 47 at time 5
DEBUG:up without down for pitch 44 at time 5
DEBUG:consecutive downs for pitch 76 at time 4 and 5
DEBUG:up without down for pitch 85 at time 5
DEBUG:up without down for pitch 41 at time 5
DEBUG:up without down for pitch 45 at time 5
DEBUG:up without down for pitch 49 at time 6
DEBUG:up without down for pitch 80 at time 6
DEBUG:up without down for pitch 44 at time 6
DE


Average composer vs critic experiment: 0.8624376890063286
ComposerLSTM(
  (embedding): Embedding(382, 10)
  (lstm): LSTM(10, 64, num_layers=2, batch_first=True, dropout=0.3)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_1): Linear(in_features=50, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=382, bias=True)
  (softmax): Softmax(dim=1)
)

Number of notes in Composer vocabulary: 382
Sequence length is completely divisible by 50
